In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pathlib
import urllib.request

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.transforms
import scipy.ndimage.measurements
import scipy.interpolate

import imageio

import pymedphys._mocks.profiles
import pymedphys._wlutz.findfield
import pymedphys._wlutz.createaxis
import pymedphys._wlutz.interppoints

In [ ]:
degrees = 20
edge_lengths = [8, 12]
penumbra = 1
centre = [15, 4]

field = pymedphys._mocks.profiles.create_rectangular_field_function(centre, edge_lengths, penumbra, degrees)
origin_field = pymedphys._mocks.profiles.create_rectangular_field_function([0,0], edge_lengths, penumbra, 0)

In [ ]:
x = np.arange(-15, 30, 0.1)
y = np.arange(-15, 15, 0.1)

xx, yy = np.meshgrid(x, y)

zz = field(xx, yy)
plt.pcolormesh(xx, yy, zz)
plt.axis('equal')

In [ ]:
mock_initial_centre = pymedphys._wlutz.findfield.initial_centre(x, y, zz)
mock_initial_centre

In [ ]:
xx_left_right, yy_left_right, xx_top_bot, yy_top_bot = pymedphys._wlutz.interppoints.define_penumbra_points_at_origin(edge_lengths, penumbra)

plt.pcolormesh(xx, yy, zz)
plt.scatter(xx_left_right, yy_left_right, s=1)
plt.scatter(xx_top_bot, yy_top_bot, s=1)

In [ ]:
transform = matplotlib.transforms.Affine2D()
transform.rotate_deg(-degrees)
transform.translate(*centre)

In [ ]:
def apply_transform(xx, yy, transform):
    xx_flat = np.ravel(xx)
    transformed = transform @ np.vstack([xx_flat, np.ravel(yy), np.ones_like(xx_flat)])
    
    xx_transformed = transformed[0]
    yy_transformed = transformed[1]
    
    xx_transformed.shape = xx.shape
    yy_transformed.shape = yy.shape
    
    return xx_transformed, yy_transformed

In [ ]:
xx_left_right_transformed, yy_left_right_transformed = apply_transform(xx_left_right, yy_left_right, transform)
xx_top_bot_transformed, yy_top_bot_transformed = apply_transform(xx_top_bot, yy_top_bot, transform)

In [ ]:
plt.pcolormesh(xx, yy, zz)
plt.scatter(xx_left_right_transformed, yy_left_right_transformed, s=1)
plt.scatter(xx_top_bot_transformed, yy_top_bot_transformed, s=1)

In [ ]:
np.allclose(
    origin_field(xx_left_right, yy_left_right),
    field(xx_left_right_transformed, yy_left_right_transformed)
)

In [ ]:
np.allclose(
    origin_field(xx_top_bot, yy_top_bot),
    field(xx_top_bot_transformed, yy_top_bot_transformed)
)